<a href="https://colab.research.google.com/github/YuanYuanBingZi/Blockchain-and-Cryptocurrency/blob/main/CS472_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten

In [ ]:
# grab the data from google drive
from google.colab import drive
drive.mount('/content/drive')

# Specify a path variable (project path)
proj_path = 'drive/MyDrive/stress/'

# Load the data into a DataFrame
df = pd.read_csv(proj_path + 'Stress.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


In [ ]:
not_used_cols = ['subreddit','post_id','sentence_range','confidence','social_timestamp']
df1 = df.drop(not_used_cols,axis=1)
df1.sample(3)

,text,label
1838,Her name was Chinook. I wasn't there. Edit--th...,0
22,Through work I have been in some dodgy situati...,1
1456,"I meant this stuff, and I think I still sort o...",1


In [ ]:

# Bag Of Words
# Step 1: Preprocess the text data
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(texts):
    new_texts = []
    for text in texts:
        # Remove punctuation and special symbols
        text = text.translate(str.maketrans('', '', string.punctuation))
    
       # Convert text to lowercase
        text = text.lower()
    
       # Tokenize the text into words
        tokens = word_tokenize(text)
    
       # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    
       # Join the tokens back into a single string
        preprocessed_text = ' '.join(tokens)
        new_texts.append(preprocessed_text)
    return new_texts


texts = df1['text']
preprocessed_texts = preprocess_text(texts)

# Step 2: Tokenization and Create Vocabulary
tokenized_documents = []
vocabulary = set()

for text in preprocessed_texts:
    # Tokenize the text into words
    tokens = text.split()
    tokenized_documents.append(tokens)
    
    # Update the vocabulary
    vocabulary.update(tokens)

vocabulary = list(vocabulary)

# Print vocabulary
print("\nVocabulary:")
print(vocabulary)

# Step 4: Vectorization
vectorizer = CountVectorizer(vocabulary=vocabulary, binary=True, lowercase=False)
bow_matrix = vectorizer.fit_transform([' '.join(text) for text in tokenized_documents])

# Step 5: Representation
bow_representation = bow_matrix.toarray()
print(bow_representation)








[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Vocabulary:
['2012', 'gave', 'carolina', 'sustain', 'wages', 'american', 'usually', 'starters', 'driver', 'expressway', 'matts', 'headboard', 'journey', 'hoard', 'opinion', 'nine', 'rims', 'vindictive', 'signing', 'nevertheless', 'begin', 'homes', 'afterward', 'gotama', 'foodgrocery', 'urging', 'insession', 'houses', 'limbs', 'crept', 'confronting', 'investigator', 'stripped', 'sos', 'tricyclic', 'numb', 'accent', 'arguing', 'dude', 'matters', '150lb', 'initial', 'demonic', 'barely', 'exclaim', 'worrisomely', 'plainly', 'negligence', 'earphones', '15', 'grief', 'extremly', 'levelheaded', 'escalated', 'anothers', 'attention', 'apologised', '2002', 'sculpted', 'feel', 'precautions', 'sociopath', 'inappropriate', 'harder', 'definitely', 'explosives', 'drs', 'degradation', 'rehabilitation', 'ambitions', 'satisfied', 'snapping', 'disbelief', '180', 'relapsed', 'diagnoses', 'intimidated', 'max', 'beneficial', '1400', 'broward', 'brought', 'ground', 'moment', 'pelvic', 'everythinggoals', 'ro

In [ ]:
# Define the target variable
target = df1['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bow_representation, target, test_size=0.2, random_state=42)

In [ ]:
# ML method 1: Logistic Regression based on bag of words

# Initialize and train a Logistic Regression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = logistic_regression_model.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, predictions)
print("\nAccuracy:", accuracy)


Accuracy: 0.721830985915493


In [ ]:
# ML method 2: SVM based on bag of words
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = svm_model.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, predictions)
print("\nAccuracy:", accuracy)


Accuracy: 0.727112676056338


In [ ]:
# Word2vec
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Example text corpus
corpus = df1['text']
# Preprocessing and tokenization
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    return filtered_tokens

# Prepare sentences
sentences = [preprocess_text(text) for text in corpus]

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Test the trained model
word_vector = model.wv["books"]
similar_words = model.wv.most_similar("reading")

print("Vector representation of 'books':", word_vector)
print("Similar words to 'reading':", similar_words)


Vector representation of 'books': [-0.08002656  0.17007308  0.05521729 -0.01475411  0.00905708 -0.30003595
  0.06747707  0.37640575 -0.16333991 -0.11481463 -0.09426707 -0.2625632
  0.03711646  0.05900105  0.06935038 -0.15986983  0.02378281 -0.18611154
  0.02980897 -0.42115486  0.06988806  0.12621722  0.11994407 -0.06180138
 -0.05931115 -0.03984793 -0.15921697 -0.17680228 -0.19122322 -0.00872548
  0.20191024  0.04614825  0.05389585 -0.13201368 -0.11305036  0.20005879
 -0.02048639 -0.13613708 -0.15019688 -0.31339332 -0.0230835  -0.17202863
 -0.03808822 -0.01169557  0.15319823 -0.11410078 -0.15141761 -0.02986388
  0.05899724  0.18109569  0.08341216 -0.16589652 -0.05883674 -0.03137144
 -0.09122897  0.0898033   0.10293049 -0.02305632 -0.22115962  0.05236293
  0.05795055  0.09808716 -0.00907024 -0.04476953 -0.23510881  0.13494305
  0.06686618  0.10283169 -0.24718319  0.19707315 -0.12563086  0.13079157
  0.21134914 -0.02664568  0.18828622  0.09629044 -0.0325421  -0.04434596
 -0.17620179  0.10

In [ ]:
# Prepare training data
X = np.array([np.mean([model.wv[word] for word in sentence], axis=0) for sentence in sentences])
y = np.array([label for label in df1['label']])  

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# ML method 1: Logistic Regression
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Initialize and train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict labels for test data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6073943661971831


In [ ]:
# ML method 2: SVM 
svm = SVC()
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.6003521126760564


In [ ]:
# ML method 3: CNN
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Prepare training data
sentences = [preprocess_text(text) for text in corpus]

# Determine the maximum sequence length
max_length = max(len(sent) for sent in sentences)

# Define the number of classes
num_classes = 2

# Convert sentences to sequences of word indices
sequences = [[model.wv.key_to_index[word] for word in sent if word in model.wv.key_to_index] for sent in sentences]


# Pad sequences to a fixed length
sequences_padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Prepare training data
X = sequences_padded
y = np.array([label for label in df1['label']])  # Assuming you have corresponding labels for each sentence

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)  # Check the shape of X_train

# Reshape the data
input_shape = (X_train.shape[1],)  # Shape of Word2Vec embeddings

# Define the CNN Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=model.wv.vectors.shape[0], output_dim=model.wv.vectors.shape[1],
                              weights=[model.wv.vectors], input_length=max_length, trainable=False),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.1)

# Predict labels for test data
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


AttributeError: ignored